# Playing with Boxes

In this notebook, we will perform the following tasks in **pytorch as well as tensorflow**:
1. Create a tensor representation of a box
2. Represent specific boxes using your tensor representation
3. Create a function which calculates the intersection of two boxes
3. Create a differentiable function to compute "soft" volumes of boxes
4. Train one box to contain another


This notebook is intended to be self-contained, but you may find it beneficial to consult Section 3.1 to 3.4 in [Representing Joint Hierarchies with Box Embeddings](https://openreview.net/pdf?id=J246NSqR_l).

## 1. Create a box parameterization

A "box" is a product of intervals in $\mathbb R^n$, i.e.

$$X = \prod_{i=1}^d [x_{m,i}, x_{M,i}], \quad \text{where} \quad x_{M,i} \ge x_{m,i} \quad \text{ for all } \quad i.$$

However you would like, create a way of storing parameters which represent a box. Crucially, your parameterization should conform to the requirement above, namely any setting of your parameterization should represent a box that has positive side-lengths in each dimension, and you should be able to represent any box in $\mathbb R^n$. Furthermore, you should be able to return the min and max coordinates (i.e. the $x_{m,i}, x_{M, i}$ above) for your box. 

**Hint**: It might be benificial to create a "wrapper" class that holds the tensor(s) corresponding to the parameters of the box.

In [40]:
import tensorflow as tf
import numpy as np

def box_class(x_min,x_max):
    return tf.Variable([x_min,x_max],dtype=tf.float32)

print(box_class([0,0],[1,1]))

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 0.],
       [1., 1.]], dtype=float32)>


## 2. Represent Specific Boxes
Using your chosen parameterization, represent a box `x` in $\mathbb R^{100}$ which has side-lengths $2$ centered at $(-1,\ldots, -1)$. 
Create another box `y` with min-coordinate $(1, \frac 1 2, \frac 1 3, \ldots, \frac 1 {100})$, and max-coordinate $(3 - \frac 1 {100}, 3 - \frac 1 {99}, \ldots, 2)$.


In [44]:
# Represent a specific box
box_length=2
diff=box_length/2

x_min=[-1-diff for _ in range(100)]
x_max=[-1+diff for _ in range(100)]

x = box_class(x_min,x_max)

y_min=[]
y_max=[]

for i in range(1,101):
    y_min.append(1/i)
    y_max.append(3-(1/(101-i)))

y = box_class(y_min,y_max)

## 3. Create an Intersection Function

The box intersection operation takes two boxes as input and returns their intersection. In one-dimension, this is as follows:

$$\text{Int}([x_m, x_M], [y_m, y_M]) = (\max(x_m, y_m), \min(x_M, y_M)).$$

Generalize this to $n$-dimensions, and write an intersection function which operates on your box parameterization.

Take the intersection of the two boxes `x` and `y` you created in step 2, and display the min and max coordinates of the intersection.

In [45]:
# Intersection Function
def box_intersection(box1,box2):
    return (tf.math.maximum(box1[0],box2[0]),tf.math.minimum(box1[1],box2[1]))

## 4. Create a "Soft" Volume Function
The "softplus volume" of a box is defined as

$$\text{SoftVol}[X] = \prod_{i=1}^d \log(1 + \exp(x_{M,i} - x_{m,i})).$$

Create this volume function, and attempt to use it to determine which of the boxes `x` and `y` from part (2) are larger. What issues do you encounter? Can you mitigate these issues?

In [46]:
# Soft Volume Function
def box_softvolume(box):
    return tf.math.reduce_prod(tf.math.log(1+tf.math.exp(box[1]-box[0])))

print(box_softvolume(x)) # returns 5.96e32
print(box_softvolume(y)) # returns inf

# Issue encountered is that the values returned are very large
# With such large values it becomes difficult to compare volume
# To mitigate this issue to certain extent we can derive intuition from gradient clipping
# By choosing a hyperparameter 'c' we clip everything exceeding c to certain value d
# This will help normalize the higher valued volumes for the boxes
# Additionally there are also functions like tf.clip_by_global_norm/tf.clip_by_norm that can be looked into

tf.Tensor(5.9604763e+32, shape=(), dtype=float32)
tf.Tensor(inf, shape=(), dtype=float32)


## 5. Training Boxes
We say box $X$ contains box $Y$ if, for each dimension $i$, we have
$$x_{m,i} < y_{m,i} < y_{M,i} < x_{M,i}.$$

Freezing the coordinates of box `x` from earlier, use gradient-descent to train box `y` such that it is contained in `x`.

**Hint:** Visualize typical boxes in 2-dimensions, and note that if a box $Y$ were contained in $X$ then $\text{Vol}(\text{Int}(X, Y)) = \text{Vol}(Y)$.

**Hint:** You can use/invent any loss function as long as it has a local minima that corresponds to satisfying the inequality given above.

In [ ]:
# Training boxes
def loss_function():
    return y[0]-x[0] + x[1]-y[1]

opt = tf.keras.optimizers.SGD(learning_rate=0.1)
for i in range(50):
	opt.minimize(loss_function, var_list=[y[0],y[1]])